# Open DOTA

## Acquiring Match Data

In [5]:
# import relevant libraries
import pandas as pd
import json
import requests
import math
import time

url_sql = "https://api.opendota.com/api/explorer"

In [2]:
test = "SELECT DISTINCT patch FROM matches m JOIN match_patch p ON m.match_id=p.match_id ORDER BY patch ASC"

params = {"sql": test}

response = requests.get(url=url_sql, params=params)

print(response)

data = response.json()

df = pd.DataFrame(data["rows"])

df

<Response [200]>


,patch
0,6.74
1,6.75
2,6.76
3,6.77
4,6.78
5,6.79
6,6.80
7,6.81
8,6.82
9,6.83


In [3]:
patches_list = df["patch"].iloc[23:].tolist()

patches_list

['7.08',
 '7.09',
 '7.10',
 '7.11',
 '7.12',
 '7.13',
 '7.14',
 '7.15',
 '7.16',
 '7.17',
 '7.18',
 '7.19',
 '7.20',
 '7.21',
 '7.22',
 '7.23',
 '7.24',
 '7.25',
 '7.26',
 '7.27',
 '7.28',
 '7.29',
 '7.30',
 '7.31',
 '7.32',
 '7.33',
 '7.34',
 '7.35']

Lets attempt to injest the entire database in our chosen range by patches to break it down, since the entire request leads to a timeout

In [4]:
# use api to collect all the patches of match data and append to a master list of matches

df_matches = pd.DataFrame()

for patch in patches_list:
  # get current patches data
  sql_string = f"SELECT * FROM matches m JOIN match_patch p ON m.match_id=p.match_id WHERE patch = '{patch}'"

  params = {"sql": sql_string}

  response = requests.get(url=url_sql, params=params)

  print(response)

  data = response.json()

  # add it to a temporary df
  df_temp = pd.DataFrame(data["rows"])
  # concat it with the master df
  df_matches = pd.concat([df_matches, df_temp])

df_matches

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [400]>


KeyError: 'rows'

This also lead to a timeout unfortunately, with the last successful patch being 7.20

In [152]:
df_matches[df_matches["patch"] == '7.20']

,match_id,match_seq_num,radiant_win,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,cluster,...,teamfights,version,cosmetics,radiant_score,dire_score,draft_timings,series_id,series_type,replay_salt,patch
0,4228171098,3664371293,False,1542652288,2368,1280,1982,48,63,134,...,"[{'start': 690, 'end': 728, 'last_death': 713,...",21.0,"{'647': 0, '4670': 132, '4674': 131, '5644': 0...",22,37,"[{'order': 1, 'pick': False, 'active_team': 2,...",269147.0,1.0,6.251827e+08,7.20
1,4229090740,3665151308,True,1542703002,2266,1982,256,63,48,172,...,"[{'start': 543, 'end': 587, 'last_death': 572,...",21.0,"{'647': 1, '4377': 131, '6353': 2, '6354': 2, ...",32,17,"[{'order': 1, 'pick': False, 'active_team': 2,...",269304.0,1.0,1.816050e+09,7.20
2,4229137839,3665200029,False,1542704936,2534,0,1956,0,63,171,...,"[{'start': 222, 'end': 264, 'last_death': 249,...",21.0,"{'647': 2, '4349': 130, '4395': 1, '4401': 1, ...",27,38,"[{'order': 1, 'pick': False, 'active_team': 2,...",269310.0,1.0,8.782618e+08,7.20
3,4229181752,3665242415,True,1542706605,2950,1828,4,63,3,172,...,"[{'start': 187, 'end': 233, 'last_death': 218,...",21.0,"{'647': 0, '5156': 131, '5412': 3, '5810': 2, ...",42,38,"[{'order': 1, 'pick': False, 'active_team': 2,...",269304.0,1.0,2.024758e+09,7.20
4,4229252732,3665270006,True,1542709354,1425,2047,1796,63,51,171,...,"[{'start': 187, 'end': 236, 'last_death': 221,...",21.0,"{'647': 129, '4811': 4, '4812': 4, '4813': 4, ...",23,15,"[{'order': 1, 'pick': False, 'active_team': 2,...",269310.0,1.0,1.145608e+09,7.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,4372442258,3784929129,True,1548768672,2247,1591,0,15,0,156,...,"[{'start': 523, 'end': 562, 'last_death': 547,...",21.0,"{'4011': 1, '4033': 1, '4119': 1, '4121': 1, '...",44,26,"[{'order': 1, 'pick': False, 'active_team': 2,...",0.0,0.0,2.629997e+08,7.20
2504,4372554005,3784988285,True,1548770450,1805,1846,1536,63,11,153,...,"[{'start': 397, 'end': 445, 'last_death': 430,...",21.0,"{'647': 1, '4063': 128, '4078': 128, '4547': 1...",34,11,"[{'order': 1, 'pick': False, 'active_team': 2,...",291524.0,1.0,1.627079e+09,7.20
2505,4372639875,3784989547,True,1548771738,1101,2047,1926,63,51,155,...,"[{'start': 87, 'end': 141, 'last_death': 126, ...",21.0,"{'4777': 130, '5263': 4, '5949': 1, '5953': 1,...",50,7,"[{'order': 1, 'pick': False, 'active_team': 2,...",291526.0,0.0,1.791935e+09,7.20
2506,4372762646,3785092834,True,1548773753,1735,1855,1972,63,63,154,...,"[{'start': 394, 'end': 436, 'last_death': 421,...",21.0,"{'647': 4, '4019': 129, '4895': 2, '4896': 2, ...",49,17,"[{'order': 1, 'pick': False, 'active_team': 2,...",291524.0,1.0,1.667684e+09,7.20


patch 7.21 seems to be too large to download in 1 chunk, lets see

In [5]:
sql_string = f"SELECT COUNT(*) FROM matches m JOIN match_patch p ON m.match_id=p.match_id WHERE patch = '7.21'"

params = {"sql": sql_string}

response = requests.get(url=url_sql, params=params)

print(response)

data = response.json()

data["rows"]

<Response [200]>


[{'count': 5767}]

Patch 7.21 lasted from the end of January till the end of May 2019, and is the largest set of data we've tried to ingest yet, lets skip it for now and continue our previous method

In [6]:
# slice the list
patches_list[14:]

['7.22',
 '7.23',
 '7.24',
 '7.25',
 '7.26',
 '7.27',
 '7.28',
 '7.29',
 '7.30',
 '7.31',
 '7.32',
 '7.33',
 '7.34',
 '7.35']

In [7]:
# slice the list to begin from 7.22
for patch in patches_list[15:]:
  # get current patches data
  sql_string = f"SELECT * FROM matches m JOIN match_patch p ON m.match_id=p.match_id WHERE patch = '{patch}'"

  params = {"sql": sql_string}

  response = requests.get(url=url_sql, params=params)

  print(response)

  data = response.json()

  # add it to a temporary df
  df_temp = pd.DataFrame(data["rows"])
  # concat it with the master df
  df_matches = pd.concat([df_matches, df_temp])

df_matches

<Response [200]>
<Response [200]>
<Response [200]>
<Response [400]>


KeyError: 'rows'

Since this method is hit or miss and we'd prefer to be able to take every single entry in 1 block of code, lets update our methodology using the dataframe.

Instead of going through the data by patch, lets query based on match_id and update the range of ids in the query dynamically in a loop till we get to the end of the list

In [8]:
print(df_matches["match_id"].min())

sql_string = f"SELECT MAX(match_id) FROM matches"

params = {"sql": sql_string}

response = requests.get(url=url_sql, params=params)

data = response.json()

print(data["rows"])

3710610589
[{'max': 7721721938}]


In [9]:
match_id_range = int(data["rows"][0]["max"]) - df_matches["match_id"].min()
match_id_range

4011111349

The range is 4bn+ but many matches are not in the list, now if we take the entire count of the set to see how many values will be in our dataframe at the end for our chosen range

In [10]:
print(df_matches["match_id"].min())

sql_string = f"SELECT COUNT(*) FROM matches WHERE start_time > 1542652288"

params = {"sql": sql_string}

response = requests.get(url=url_sql, params=params)

data = response.json()

print(data["rows"])

3710610589
[{'count': 124603}]


We can now have a look at our dataframe so far to see how many values can be retrieved from a successful query max

In [11]:
df_matches["patch"].value_counts()

patch
7.24    3108
7.19    2981
7.23    2972
7.20    2508
7.25    2237
7.11     521
7.18     477
7.08     463
7.13     382
7.16     361
7.15     344
7.12     307
7.09     261
7.17     238
7.14     216
7.10     202
Name: count, dtype: int64

Ok, seems the max is around 3k, lets make that 2,500 to be safe and divide our range of ids by 2,500 to get a range of matches to retrieve each iteration

In [12]:


# set max match_id
match_id_max = df_matches["match_id"].min() + match_id_range
match_id_min = df_matches["match_id"].min()

# roughly how many match_ids between each actual recorded record
per_match_id = math.floor(match_id_range / 124601)
# roughly how many records per 2500 match_ids
match_id_steps = math.floor(per_match_id * 2500)

i = match_id_min
csv_int = 101

while i < match_id_max:
  j = i + match_id_steps

  print(f"attempting to retrieve {i} to {j} records")
  # get current patches data
  sql_string = f"SELECT * FROM matches m JOIN match_patch p ON m.match_id=p.match_id WHERE m.match_id BETWEEN {i} AND {j}"

  params = {"sql": sql_string}

  response = requests.get(url=url_sql, params=params)

  # print(response)

  data = response.json()

  # add it to a temporary df
  df_temp = pd.DataFrame(data["rows"])

  # drop rows we won't use in our analysis
  df_temp = df_temp.drop(columns=["objectives", "chat", "engine", "human_players", "lobby_type", "version", "replay_salt", "match_seq_num"])
  # concat it with the master df
  df_temp.to_csv(f"data/raw/matches_raw_{csv_int}.csv", index=False, sep='\t')
  df_temp.to_json(f"data/raw/matches_raw_{csv_int}.json", index=False)

  i = j
  csv_int += 1
  time.sleep(2)


attempting to retrieve 3710610589 to 3791088089 records
attempting to retrieve 3791088089 to 3871565589 records
attempting to retrieve 3871565589 to 3952043089 records
attempting to retrieve 3952043089 to 4032520589 records
attempting to retrieve 4032520589 to 4112998089 records
attempting to retrieve 4112998089 to 4193475589 records
attempting to retrieve 4193475589 to 4273953089 records
attempting to retrieve 4273953089 to 4354430589 records
attempting to retrieve 4354430589 to 4434908089 records
attempting to retrieve 4434908089 to 4515385589 records
attempting to retrieve 4515385589 to 4595863089 records
attempting to retrieve 4595863089 to 4676340589 records
attempting to retrieve 4676340589 to 4756818089 records
attempting to retrieve 4756818089 to 4837295589 records
attempting to retrieve 4837295589 to 4917773089 records


KeyError: 'rows'

this was unfortunate, have to assume another timeout, lets try it again with 75% of the entries per loop from where we left off

In [13]:
# set min match_id and range
match_id_min = 4837295589
match_id_range = match_id_max - match_id_min

i = match_id_min
csv_int = 115

while i < match_id_max:
  j = math.floor(i + (match_id_steps * 0.75))

  print(f"attempting to retrieve {i} to {j} records")
  # get current patches data
  sql_string = f"SELECT * FROM matches m JOIN match_patch p ON m.match_id=p.match_id WHERE m.match_id BETWEEN {i} AND {j}"

  params = {"sql": sql_string}

  response = requests.get(url=url_sql, params=params)

  # print(response)

  data = response.json()

  # add it to a temporary df
  df_temp = pd.DataFrame(data["rows"])

  # drop rows we won't use in our analysis
  df_temp = df_temp.drop(columns=["objectives", "chat", "engine", "human_players", "lobby_type", "version", "replay_salt", "match_seq_num"])
  # concat it with the master df
  df_temp.to_csv(f"data/raw/matches_raw_{csv_int}.csv", index=False, sep='\t')
  df_temp.to_json(f"data/raw/matches_raw_{csv_int}.json", index=False)

  i = j
  csv_int += 1
  time.sleep(2)

attempting to retrieve 4837295589 to 4897653714 records
attempting to retrieve 4897653714 to 4958011839 records
attempting to retrieve 4958011839 to 5018369964 records
attempting to retrieve 5018369964 to 5078728089 records
attempting to retrieve 5078728089 to 5139086214 records


We have imported the entire range into csv files

Now lets reimport the entire data range into a pandas dataframe and remove the duplicate rows

In [7]:
import os

# Directory path
directory = "data/raw/"

# Get list of all files in the directory
files = os.listdir(directory)

#declare an empty df
df = pd.DataFrame()

for file in files:
  if ".csv" in file:
    temp_df = pd.read_csv(directory+file, sep="\t")
    df = pd.concat([df, temp_df])

df.head()

,match_id,radiant_win,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,cluster,first_blood_time,...,radiant_gold_adv,radiant_xp_adv,teamfights,cosmetics,radiant_score,dire_score,draft_timings,series_id,series_type,patch
0,7070560119,False,1679387808,2134,1830.0,2020.0,63.0,63.0,272,116.0,...,"[0, -332, -811, -356, -479, -338, -151, 402, -...","[0, 68, -179, 255, -71, -472, -232, 23, -596, ...","[{'start': 497, 'end': 537, 'last_death': 522,...",{'684': 2},29.0,45.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762547.0,1.0,7.32
1,7070580147,True,1679389590,2608,1590.0,1796.0,63.0,63.0,184,148.0,...,"[0, -439, -423, 262, -376, -246, -719, -466, -...","[0, 1, 15, 429, -23, -228, -700, -60, -519, -6...","[{'start': 484, 'end': 528, 'last_death': 513,...","{'630': 132, '647': 130, '5644': 131, '6282': ...",35.0,23.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762548.0,1.0,7.32
2,7070586665,False,1679389694,2831,768.0,2036.0,60.0,63.0,271,247.0,...,"[0, -122, 497, 574, 540, 231, 608, 281, 415, -...","[0, -89, 2, -86, -23, -248, -73, 307, 406, -95...","[{'start': 644, 'end': 681, 'last_death': 666,...","{'647': 129, '4367': 129, '4749': 4, '4837': 1...",33.0,42.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762554.0,1.0,7.32
3,7070587273,True,1679389735,2529,1982.0,0.0,63.0,0.0,154,184.0,...,"[0, 254, 47, 630, 864, 1847, 2327, 3063, 2898,...","[0, -243, -390, 3, -131, 166, 958, 701, -56, 2...","[{'start': 978, 'end': 1025, 'last_death': 101...","{'647': 0, '5965': 132, '6694': 132, '6786': 0...",32.0,13.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762563.0,1.0,7.32
4,7070587828,False,1679389824,1579,1540.0,2039.0,3.0,63.0,273,286.0,...,"[-100, -498, -545, -856, -589, -867, -659, -14...","[-50, -21, -35, -268, -304, -517, -493, -1173,...","[{'start': 901, 'end': 944, 'last_death': 929,...","{'602': 130, '4054': 130, '5442': 1, '5443': 1...",18.0,30.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762564.0,1.0,7.32


In [13]:
df = df.drop_duplicates()
df

,match_id,radiant_win,start_time,duration,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,cluster,first_blood_time,...,radiant_gold_adv,radiant_xp_adv,teamfights,cosmetics,radiant_score,dire_score,draft_timings,series_id,series_type,patch
0,7070560119,False,1679387808,2134,1830.0,2020.0,63.0,63.0,272,116.0,...,"[0, -332, -811, -356, -479, -338, -151, 402, -...","[0, 68, -179, 255, -71, -472, -232, 23, -596, ...","[{'start': 497, 'end': 537, 'last_death': 522,...",{'684': 2},29.0,45.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762547.0,1.0,7.32
1,7070580147,True,1679389590,2608,1590.0,1796.0,63.0,63.0,184,148.0,...,"[0, -439, -423, 262, -376, -246, -719, -466, -...","[0, 1, 15, 429, -23, -228, -700, -60, -519, -6...","[{'start': 484, 'end': 528, 'last_death': 513,...","{'630': 132, '647': 130, '5644': 131, '6282': ...",35.0,23.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762548.0,1.0,7.32
2,7070586665,False,1679389694,2831,768.0,2036.0,60.0,63.0,271,247.0,...,"[0, -122, 497, 574, 540, 231, 608, 281, 415, -...","[0, -89, 2, -86, -23, -248, -73, 307, 406, -95...","[{'start': 644, 'end': 681, 'last_death': 666,...","{'647': 129, '4367': 129, '4749': 4, '4837': 1...",33.0,42.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762554.0,1.0,7.32
3,7070587273,True,1679389735,2529,1982.0,0.0,63.0,0.0,154,184.0,...,"[0, 254, 47, 630, 864, 1847, 2327, 3063, 2898,...","[0, -243, -390, 3, -131, 166, 958, 701, -56, 2...","[{'start': 978, 'end': 1025, 'last_death': 101...","{'647': 0, '5965': 132, '6694': 132, '6786': 0...",32.0,13.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762563.0,1.0,7.32
4,7070587828,False,1679389824,1579,1540.0,2039.0,3.0,63.0,273,286.0,...,"[-100, -498, -545, -856, -589, -867, -659, -14...","[-50, -21, -35, -268, -304, -517, -493, -1173,...","[{'start': 901, 'end': 944, 'last_death': 929,...","{'602': 130, '4054': 130, '5442': 1, '5443': 1...",18.0,30.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",762564.0,1.0,7.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,5802838369,False,1611601072,2953,1924.0,1972.0,63.0,63.0,136,194.0,...,"[0, -3, 18, -256, -619, -672, -927, -717, -112...","[0, -159, 17, -244, -423, -739, -900, -991, -1...","[{'start': 863, 'end': 917, 'last_death': 902,...","{'630': 0, '647': 132, '4816': 1, '4817': 1, '...",34.0,41.0,"[{'order': 1, 'pick': False, 'active_team': 2,...",523411.0,1.0,7.28
1835,5802906113,True,1611605127,2158,1847.0,1792.0,63.0,48.0,251,126.0,...,"[0, 22, -68, 172, 430, 682, 780, 723, 1168, 20...","[0, 114, 125, 362, 440, 463, 121, -734, 234, 4...","[{'start': 348, 'end': 382, 'last_death': 367,...","{'602': 0, '647': 131, '4665': 129, '4707': 0,...",43.0,13.0,"[{'order': 1, 'pick': False, 'active_team': 2,...",523435.0,1.0,7.28
1836,5802943495,True,1611607163,2187,1983.0,1536.0,63.0,2.0,136,77.0,...,"[0, 125, 692, 769, 1390, 1314, 1634, 1127, 224...","[0, 35, 339, 6, 446, 300, 808, 51, 1259, 1801,...","[{'start': 513, 'end': 551, 'last_death': 536,...","{'602': 128, '630': 132, '647': 131, '4054': 1...",39.0,13.0,"[{'order': 1, 'pick': False, 'active_team': 3,...",523411.0,1.0,7.28
1837,5802966767,True,1611608664,1254,2047.0,1830.0,63.0,63.0,251,62.0,...,"[0, 531, 665, -46, 303, 372, 1067, 807, 613, 1...","[0, 131, -110, -868, -650, -536, -298, -58, -3...","[{'start': 312, 'end': 357, 'last_death': 342,...","{'602': 128, '647': 128, '4369': 1, '4394': 1,...",33.0,18.0,"[{'order': 1, 'pick': False, 'active_team': 2,...",523435.0,1.0,7.28


## Flatten the Picks and Bans

In [56]:
import re

temp_list = []

# for every entry create entries in a new dataframe that will contain all picks/bans of that match id
for index, row in df.iterrows():
    
    # Convert the value to a string and then replace single quotes with double quotes in the JSON string
    picks_bans_json = str(row["picks_bans"]).replace("'", '"')
    picks_bans_json = re.sub(r'False', 'false', picks_bans_json)
    picks_bans_json = re.sub(r'True', 'true', picks_bans_json)
    print(picks_bans_json)
    # print(picks_bans_json)  # Print the modified JSON string before parsing
    try:
        picks_bans_list = json.loads(picks_bans_json)
    except json.JSONDecodeError as e:
        print("Error:", e)
        continue

    for item in picks_bans_list:
        updated_item = dict(item)
        updated_item.update({"match_id": row['match_id']})
        print(updated_item)
        temp_list.append(updated_item)
    # print(temp_list)

# add the list of picks and bans to a df
df_picks_bans = pd.DataFrame()



[{"is_pick": false, "hero_id": 39, "team": 0, "order": 0}, {"is_pick": false, "hero_id": 86, "team": 1, "order": 1}, {"is_pick": false, "hero_id": 101, "team": 0, "order": 2}, {"is_pick": false, "hero_id": 31, "team": 1, "order": 3}, {"is_pick": true, "hero_id": 21, "team": 0, "order": 4}, {"is_pick": true, "hero_id": 9, "team": 1, "order": 5}, {"is_pick": true, "hero_id": 87, "team": 1, "order": 6}, {"is_pick": true, "hero_id": 40, "team": 0, "order": 7}, {"is_pick": false, "hero_id": 54, "team": 0, "order": 8}, {"is_pick": false, "hero_id": 6, "team": 1, "order": 9}, {"is_pick": false, "hero_id": 28, "team": 0, "order": 10}, {"is_pick": false, "hero_id": 93, "team": 1, "order": 11}, {"is_pick": false, "hero_id": 8, "team": 0, "order": 12}, {"is_pick": false, "hero_id": 114, "team": 1, "order": 13}, {"is_pick": true, "hero_id": 14, "team": 1, "order": 14}, {"is_pick": true, "hero_id": 49, "team": 0, "order": 15}, {"is_pick": true, "hero_id": 63, "team": 0, "order": 16}, {"is_pick": tr

In [43]:
print(temp_list)

[]


## Flatten the Team Fights?